In [ ]:
!pip install pyomo==6.7.1

In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
excel_file = Path("../data/ChallengeXHEC23022024.xlsx")

In [ ]:
schedule = pd.read_excel(excel_file, sheet_name=0)
clients = pd.read_excel(excel_file, sheet_name=1)
caregivers = pd.read_excel(excel_file, sheet_name=2)

In [ ]:
schedule.Prestation.unique()

In [ ]:
discard_list = [
    "ADMINISTRATION",
    "VISITE MEDICALE",
    "FORMATION",
    "COORDINATION",
    "HOMMES TOUTES MAINS",
]

In [ ]:
sched_one_day = schedule.loc[schedule.Date == "2024-01-03", :]

In [ ]:
sched_one_day.Prestation.value_counts()

In [ ]:
sched_one_day[~sched_one_day.Prestation.isin(discard_list)]

In [ ]:
sched_one_day["idx"] = sched_one_day.index

In [ ]:
from datetime import datetime

common_date = sched_one_day["Date"].iloc[0]
sched_one_day["Heure de fin"] = sched_one_day["Heure de fin"].apply(
    lambda x: datetime.combine(common_date, x)
)
sched_one_day["Heure de début"] = sched_one_day["Heure de début"].apply(
    lambda x: datetime.combine(common_date, x)
)
sched_one_day["Duration"] = (
    sched_one_day["Heure de fin"] - sched_one_day["Heure de début"]
)
sched_one_day["Duration"] = sched_one_day["Duration"].apply(
    lambda x: x.seconds // 60
)

In [ ]:
sched_one_day

In [ ]:
caregivers

In [ ]:
schedule

In [ ]:
# Import pyomo libraries
import pyomo.environ as pe
import pyomo.gdp as pyogdp

from itertools import product

In [ ]:
class CareScheduler:
    def __init__(self, excel_file):
        """
        Read cargiver and session data into Pandas DataFrames
        Args:
            excel_file (str): path to data in excel format
        """
        try:
            clients = pd.read_excel(excel_file, sheet_name=1)
            sessions = pd.read_excel(excel_file, sheet_name=0)

            # NOTE: remove later
            sessions = sessions.loc[sessions.Date == "2024-01-03", :]
            sessions["idx"] = sessions.index

            # create duration
            common_date = sessions["Date"].iloc[0]
            sessions["Heure de fin"] = sessions["Heure de fin"].apply(
                lambda x: datetime.combine(common_date, x)
            )
            sessions["Heure de début"] = sessions["Heure de début"].apply(
                lambda x: datetime.combine(common_date, x)
            )
            sessions["Duration"] = (
                sessions["Heure de fin"] - sessions["Heure de début"]
            )
            sessions["Duration"] = sessions["Duration"].apply(
                lambda x: x.seconds // 60
            )

            # discard
            discard_list = [
                "ADMINISTRATION",
                "VISITE MEDICALE",
                "FORMATION",
                "COORDINATION",
                "HOMMES TOUTES MAINS",
            ]
            sessions = sessions[~sessions.Prestation.isin(discard_list)]

            sessions = sessions.merge(clients, how="left", on="ID Client")
            sessions = sessions.drop(columns="ID Intervenant")
            self.df_sessions = sessions
        except FileNotFoundError:
            print("Session data not found.")

        try:
            self.df_cargeivers = pd.read_excel(excel_file, sheet_name=2)
        except FileNotFoundError:
            print("Caregiver data not found")

        self.model = self.create_model()

    def _generate_case_durations(self):
        """
        Generate mapping of cases IDs to case time for the procedure
        Returns:
            (dict): dictionary with CaseID as key and median case time (mins) for procedure as value
        """
        return pd.Series(
            self.df_sessions["Duration"].values, index=self.df_sessions["idx"]
        ).to_dict()

    def _generate_disjunctions(self):
        """Returns:
        disjunctions (list): list of tuples containing disjunctions
        """
        cases = self.df_sessions["idx"].to_list()
        cargivers = self.df_cargeivers["ID Intervenant"].to_list()
        disjunctions = []
        for (case1, case2, cargiver) in product(cases, cases, cargivers):
            if (case1 != case2) and (
                case2,
                case1,
                cargiver,
            ) not in disjunctions:
                disjunctions.append((case1, case2, cargiver))

        return disjunctions

    def create_model(self):
        model = pe.ConcreteModel()

        # List of case IDs in home care client needs list
        model.CASES = pe.Set(initialize=self.df_sessions["idx"].tolist())
        # List of potential caregiver IDs
        model.CAREGIVERS = pe.Set(
            initialize=self.df_cargeivers["ID Intervenant"].tolist()
        )
        # List of tasks - all possible (caseID, caregiverID) combination
        model.TASKS = pe.Set(
            initialize=model.CASES * model.CAREGIVERS, dimen=2
        )
        # The duration (expected case time) for each operation
        model.CASE_DURATION = pe.Param(
            model.CASES, initialize=self._generate_case_durations()
        )

        # Decision Variables
        ub = 1440  # minutes in a day
        model.M = pe.Param(initialize=1e3 * ub)  # big M

        # Binary flag, 1 if case is assigned to session, 0 otherwise
        model.SESSION_ASSIGNED = pe.Var(model.TASKS, domain=pe.Binary)
        # Start time of a case
        model.CASE_START_TIME = pe.Var(
            model.TASKS, bounds=(0, ub), within=pe.PositiveReals
        )
        # Downtime of a caregiver caregiver
        model.DOWN_TIME = pe.Var(
            model.CAREGIVERS, bounds=(0, ub), within=pe.PositiveIntegers
        )

        # model.DISJUNCTIONS = pe.Set(initialize=self._generate_disjunctions(), dimen=3)

        # Objective
        def objective_function(model):
            return pe.summation(model.DOWN_TIME)
            # return sum([model.SESSION_ASSIGNED[case, session] for case in model.CASES for session in model.SESSIONS])

        model.OBJECTIVE = pe.Objective(
            rule=objective_function, sense=pe.minimize
        )

        # Constraints
        # TODO: get minimum working hours
        # Case start time must be after 8 AM
        # def case_start_time(model, case, session):
        #     return model.CASE_START_TIME[case, session] >= self.df_sessions["Heure de début"].min()
        # model.CASE_START = pe.Constraint(model.TASKS, rule=case_start_time)

        # TODO: create maximum working hours
        # # Case end time must be before general working hours 22 PM
        # def case_end_time(model, case, session):
        #     return model.CASE_START_TIME[case, session] + model.CASE_DURATION[case] <= self.df_sessions["Heure de fin"].max()
        # model.CASE_END_TIME = pe.Constraint(model.TASKS, rule=case_end_time)

        # Cases can be assigned to a maximum of one caregiver
        def session_assignment(model, case):
            return (
                sum(
                    [
                        model.SESSION_ASSIGNED[(case, session)]
                        for session in model.CAREGIVERS
                    ]
                )
                <= 1
            )

        model.SESSION_ASSIGNMENT = pe.Constraint(
            model.CASES, rule=session_assignment
        )

        # TODO: create bounding box deadlines
        # Constraint 4: Cases must be completed before their target deadline
        # def set_deadline_condition(model, case, session):
        #     return model.SESSION_DATES[session] <= model.CASE_DEADLINES[case] + ((1 - model.SESSION_ASSIGNED[case, session])*model.M)
        # model.APPLY_DEADLINE = pe.Constraint(model.TASKS, rule=set_deadline_condition)

        # Constraint 5: No two cases can overlap
        # def no_case_overlap(model, case1, case2, caregiver):
        #     return [model.CASE_START_TIME[case1, caregiver] + model.CASE_DURATION[case1] <= model.CASE_START_TIME[case2, caregiver] + \
        #             ((2 - model.SESSION_ASSIGNED[case1, caregiver] - model.SESSION_ASSIGNED[case2, caregiver])*model.M),
        #             model.CASE_START_TIME[case2, caregiver] + model.CASE_DURATION[case2] <= model.CASE_START_TIME[case1, caregiver] + \
        #             ((2 - model.SESSION_ASSIGNED[case1, caregiver] - model.SESSION_ASSIGNED[case2, caregiver])*model.M)]
        # model.DISJUNCTIONS_RULE = pyogdp.Disjunction(model.DISJUNCTIONS, rule=no_case_overlap)

        # pe.TransformationFactory("gdp.bigm").apply_to(model)

        return model

    def solve(self):
        # Create solver (synchronous solve)
        solver = pe.SolverFactory("cbc", executable="/opt/homebrew/bin/cbc")

        # Add solver parameters (time limit)
        options = {"seconds": 60}
        for key, value in options.items():
            solver.options[key] = value

        # Solve model (verbose)
        solver_results = solver.solve(self.model, tee=True)
        return solver_results

In [ ]:
scheduler.df_sessions

In [ ]:
scheduler = CareScheduler(excel_file)

In [ ]:
solver_results = scheduler.solve()

In [ ]:
solver_results

In [ ]:
model = scheduler.model

In [ ]:
solver = pe.SolverFactory("cbc", executable="/opt/homebrew/bin/cbc")
# Add solver parameters (time limit)
options = {"seconds": 60}
for key, value in options.items():
    solver.options[key] = value

In [ ]:
solver_results = solver.solve(model, tee=True)

In [ ]:
# List of case IDs in surgical waiting list
model.CASES = pe.Set(initialize=self.df_cases["CaseID"].tolist())
# List of sessions IDs
model.SESSIONS = pe.Set(initialize=self.df_sessions["SessionID"].tolist())

In [ ]:
def objective_function(model):
    return pe.summation(model.UTILISATION)


model.OBJECTIVE = pe.Objective(rule=objective_function, sense=pe.maximize)